In [1]:
import os
import FinanceDataReader as fdr
from tqdm import tqdm
import pandas as pd

# Pycaret 이용

## 모든 종목 예측

* 종가와 상관계수 높은 특성 : KOSDAQ(0.87), KOSPI(0.79), DOW(0.58), NASDAQ(0.50), S&P500(0.56)
* 예측 순서(종가와 상관계수 높은 특성 먼저 예측)
    * weekday, weeknumdm로 KOSDAQ 예측(종가와 상관계수 0.87)
    * weekday, weeknum, KOSDAQ으로 NASDAQ 예측(KOSDAQ과 상관계수 0.68)
    * weekday, weeknum, KOSDAQ, NASDAQ으로 S&P500 예측(NASDAQ과 상관계수 0.93)
    * weekday, weeknum, KOSDAQ, NASDAQ, S&P500으로 DOW 예측(S&P500과 상관계수 0.95)
    * weekday, weeknum, KOSDAQ, NASDAQ, S&P500, DOW로 KOSPI 예측
    * weekday, weeknum, KOSDAQ, NASDAQ, S&P500, DOW, KOSPI로 다른 종가 예측 

In [2]:
# 제출 점수 :

In [3]:
from pycaret.regression import *

In [4]:
# 외부 데이터 가져오기
df = pd.read_csv('../data/20210104_20211119_data_interpolate.csv')
df.head()

,Date,exchange_rate,kospi,kosdaq,kospi100,dow,nasdaq,sp500,미국채10년-2년,BTC,Gold,Oil,US10Y,US2Y
0,2021-01-04,1086.48,2944.45,977.62,3052.74,30223.89,12698.4,3700.65,0.8009,36499000.0,37.10,32.38,0.918,0.1171
1,2021-01-05,1086.42,2990.57,985.76,3099.42,30391.60,12819.0,3726.86,0.8320,37962000.0,37.18,33.94,0.955,0.1230
2,2021-01-06,1087.93,2968.21,981.39,3070.05,30829.40,12740.8,3748.14,0.9002,40731000.0,36.60,34.11,1.039,0.1388
3,2021-01-07,1094.28,3031.68,988.86,3139.48,31041.13,13067.5,3803.79,0.9402,44575000.0,36.48,34.53,1.081,0.1408
4,2021-01-08,1092.93,3152.18,987.79,3293.96,31097.97,13202.0,3824.68,0.9821,47241000.0,35.26,35.43,1.119,0.1369


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           230 non-null    object 
 1   exchange_rate  230 non-null    float64
 2   kospi          230 non-null    float64
 3   kosdaq         230 non-null    float64
 4   kospi100       230 non-null    float64
 5   dow            230 non-null    float64
 6   nasdaq         230 non-null    float64
 7   sp500          230 non-null    float64
 8   미국채10년-2년      230 non-null    float64
 9   BTC            230 non-null    float64
 10  Gold           230 non-null    float64
 11  Oil            230 non-null    float64
 12  US10Y          230 non-null    float64
 13  US2Y           230 non-null    float64
dtypes: float64(13), object(1)
memory usage: 25.3+ KB


In [6]:
df.columns

Index(['Date', 'exchange_rate', 'kospi', 'kosdaq', 'kospi100', 'dow', 'nasdaq',
       'sp500', '미국채10년-2년', 'BTC', 'Gold', 'Oil', 'US10Y', 'US2Y'],
      dtype='object')

# 필요한 특성만 추출

In [7]:
df = df[['Date','kosdaq','nasdaq','sp500','dow','kospi']]
df.head()

,Date,kosdaq,nasdaq,sp500,dow,kospi
0,2021-01-04,977.62,12698.4,3700.65,30223.89,2944.45
1,2021-01-05,985.76,12819.0,3726.86,30391.60,2990.57
2,2021-01-06,981.39,12740.8,3748.14,30829.40,2968.21
3,2021-01-07,988.86,13067.5,3803.79,31041.13,3031.68
4,2021-01-08,987.79,13202.0,3824.68,31097.97,3152.18


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    230 non-null    object 
 1   kosdaq  230 non-null    float64
 2   nasdaq  230 non-null    float64
 3   sp500   230 non-null    float64
 4   dow     230 non-null    float64
 5   kospi   230 non-null    float64
dtypes: float64(5), object(1)
memory usage: 10.9+ KB


In [9]:
# 종목 정보 가져오기
path = '../data'
list_name = 'Stock_List.csv'
stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list.head()

,종목명,종목코드,상장시장
0,삼성전자,5930,KOSPI
1,SK하이닉스,660,KOSPI
2,NAVER,35420,KOSPI
3,카카오,35720,KOSPI
4,삼성바이오로직스,207940,KOSPI


In [10]:
stock_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370 entries, 0 to 369
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   종목명     370 non-null    object
 1   종목코드    370 non-null    int64 
 2   상장시장    370 non-null    object
dtypes: int64(1), object(2)
memory usage: 8.8+ KB


In [11]:
# 종목 코드 6자리로 맞추기
stock_list['종목코드'] = stock_list["종목코드"].astype(str).str.zfill(6)
stock_list.head()

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI


# 종가 예측

* 1. kosdaq 예측
* 2. nasdaq
* 3. sp500
* 4. dow
* 5. kospi
* 6. 종가

In [12]:
start_date = '20210104'
end_date = '20211119'
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

## 기본 + 5개 + 종목 데이터 프레임 만들기
* 2021년 1월 4일 ~ 11월 19일

In [13]:
# 종목 주가 정보 가져오기
code = stock_list['종목코드'][0]
basic_data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
# 공휴일 포함한 평일 날짜로 데이터 셋 채우기
basic_data = pd.merge(Business_days, basic_data, how = 'outer')
# 요 번호와 주 번호 지정하여 열로 추가하기
basic_data['weekday'] = basic_data.Date.apply(lambda x : x.weekday())
basic_data['weeknum'] = basic_data.Date.apply(lambda x : x.strftime('%V'))
# 빈값 선형으로 채우기
basic_data.Close = basic_data.Close.interpolate()
# data의 Close 열 이름 종목 코드명으로 바꾸기
basic_data = basic_data.rename(columns={'Close':str(code)})
# 날짜 기준으로 외부 데이터와 주가 정보 합치기
basic_data['Date'] = basic_data['Date'].astype('str')
basic_data = pd.merge(df, basic_data, on='Date')
basic_data.head()

,Date,kosdaq,nasdaq,sp500,dow,kospi,005930,weekday,weeknum
0,2021-01-04,977.62,12698.4,3700.65,30223.89,2944.45,83000.0,0,01
1,2021-01-05,985.76,12819.0,3726.86,30391.60,2990.57,83900.0,1,01
2,2021-01-06,981.39,12740.8,3748.14,30829.40,2968.21,82200.0,2,01
3,2021-01-07,988.86,13067.5,3803.79,31041.13,3031.68,82900.0,3,01
4,2021-01-08,987.79,13202.0,3824.68,31097.97,3152.18,88800.0,4,01


In [14]:
basic_data.columns

Index(['Date', 'kosdaq', 'nasdaq', 'sp500', 'dow', 'kospi', '005930',
       'weekday', 'weeknum'],
      dtype='object')

In [15]:
len(basic_data.columns)

9

In [16]:
basic_data = basic_data[['Date', 'weekday', 'weeknum' ,'kosdaq', 'nasdaq', 'sp500', 'dow', 'kospi', '005930']]
basic_data.head()

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi,005930
0,2021-01-04,0,01,977.62,12698.4,3700.65,30223.89,2944.45,83000.0
1,2021-01-05,1,01,985.76,12819.0,3726.86,30391.60,2990.57,83900.0
2,2021-01-06,2,01,981.39,12740.8,3748.14,30829.40,2968.21,82200.0
3,2021-01-07,3,01,988.86,13067.5,3803.79,31041.13,3031.68,82900.0
4,2021-01-08,4,01,987.79,13202.0,3824.68,31097.97,3152.18,88800.0


In [17]:
data = basic_data.copy()

In [18]:
for code in tqdm(stock_list['종목코드'][1:]):
    # 종목 주가 정보 가져오기
    one_stock_data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    # 공휴일 포함한 평일 날짜로 데이터 셋 채우기
    one_stock_data = pd.merge(Business_days, one_stock_data, how = 'outer')
    # 빈값 선형으로 채우기
    one_stock_data.Close = one_stock_data.Close.interpolate()
    # data의 Close 열 이름 종목 코드명으로 바꾸기
    one_stock_data = one_stock_data.rename(columns={'Close':str(code)})
    # 날짜 기준으로 외부 데이터와 주가 정보 합치기
    one_stock_data['Date'] = one_stock_data['Date'].astype('str')
    data = pd.merge(data, one_stock_data, on='Date')

100%|██████████| 369/369 [01:21<00:00,  4.54it/s]


In [19]:
data.head()

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi,005930,000660,035420,035720,207940,005935,051910,006400,005380,068270,000270,005490,066570,028260,012330,096770,051900,017670,105560,055550,034730,036570,011200,015760,032830,018260,009150,090430,086790,251270,003670,352820,034020,010950,033780,003490,000810,010130,009540,326030,011170,030200,018880,034220,316140,009830,024110,086280,004020,097950,032640,011780,021240,000720,161390,006800,035250,071050,267250,011070,008930,002790,028050,139480,271560,241560,180640,000100,069500,000120,078930,003410,029780,028670,004990,005830,016360,128940,005387,298020,006360,020150,007070,008770,005940,112610,042660,307950,006280,008560,019170,010620,036460,047050,047810,138040,336260,081660,272210,023530,039490,047040,012750,026960,010060,088350,014680,002380,051915,000060,285130,204320,030000,282330,009240,000990,298050,004170,064350,005385,011210,012510,012450,004800,138930,153130,000080,006260,001450,000880,001440,252670,120110,017800,102110,003090,052690,011000,294870,001230,004370,073240,007310,051600,010120,069620,069960,042700,004000,102780,336370,111770,006650,000240,010780,016380,000150,009900,013890,005850,001740,139130,181710,175330,005250,185750,003000,051905,330590,192820,214980,122630,031430,273130,032350,066575,363280,298000,278530,000670,292150,161890,005300,004490,214320,091990,293490,247540,068760,263750,036490,028300,096530,035760,196170,066970,145020,253450,058470,357780,240810,064760,278280,032500,095700,108320,237690,007390,098460,048260,178920,034230,005290,025900,067160,078340,030190,056190,268600,041510,085660,035900,214150,200130,039030,213420,141080,060250,086900,206650,235980,323990,022100,140410,036540,000250,048410,086450,084850,046890,084370,950130,069080,299660,039200,166090,144510,122870,218410,067630,112040,215200,084990,298380,151910,194480,025980,064550,003380,215000,033640,078600,217270,089980,214450,140860,230360,027360,222800,082270,131290,088800,074600,293780,185490,178320,036830,084110,151860,092040,131970,048530,039840,290650,041960,065660,214370,272290,183490,035080,036930,101490,294090,053800,256840,078130,205470,058820,243070,299900,222080,052400,090460,183300,104480,033290,290510,041190,226950,092190,100090,319660,035600,038500,042000,030520,053030,267980,137400,068240,216080,013120,095660,060150,061970,095610,089970,036810,348210,330860,263720,143240,091700,067000,204270,032190,131370,321550,347860,003800,032300,122450,206640,015750,060720,043150,006730,089030,348150,319400,232140,083790,299030,049070,067310,036030,031390,121800,033310,086390,078020,102710,314130,035890,032620,220630,064260,287410,110790,099320
0,2021-01-04,0,01,977.62,12698.4,3700.65,30223.89,2944.45,83000.0,126000.0,293000.0,79483.0,829000.0,74400.0,889000.0,671000.0,207500.0,347500.0,64000.0,273000.0,142000.0,144000.0,287000.0,231000.0,1612000.0,237000.0,42450.0,31550.0,249500.0,978000.0,16550.0,26900.0,78000.0,186000.0,180000.0,203000.0,33450.0,131000.0,118000.0,156420.0,13900.0,70300.0,82100.0,21440.0,183000.0,401000.0,110000.0,154500.0,279500.0,23800.0,16500.0,19050.0,9510.0,47690.0,8620.0,190000.0,41200.0,384500.0,11850.0,151000.0,73200.0,37216.0,37400.0,9500.0,23350.0,82300.0,55300.0,192500.0,76000.0,54000.0,13500.0,151500.0,123000.0,29400.0,62300.0,79200.0,39732.0,168500.0,37800.0,6580.0,31250.0,5370.0,35100.0,42750.0,40800.0,357000.0,92800.0,213000.0,38050.0,52500.0,34400.0,82600.0,11200.0,88302.0,27850.0,138500.0,475000.0,3615.0,122000.0,48250.0,30800.0,14550.0,26900.0,9680.0,56300.0,42850.0,15602.0,101500.0,134500.0,5150.0,85100.0,33800.0,98500.0,2340.0,197000.0,199000.0,396000.0,14250.0,264211.0,62700.0,20250.0,132000.0,105500.0,51100.0,151000.0,238500.0,19650.0,91700.0,69500.0,106000.0,29800.0,75400.0,5560.0,102645.0,32000.0,72500.0,22400.0,29350.0,1385.0,2380.0,41350.0,40900.0,39680.0,53200.0,18100.0,21298.0,26300.0,8680.0,296500.0,3710.0,567000.0,29200.0,65000.0,172000.0,72000.0,18250.0,55100.0,10016.0,50800.0,31100.0,234000.0,14400.0,54600.0,12150.0,52000.0,48

In [20]:
# 총 378열
len(data.columns)

378

## 훈련, 테스트 셋 나누기

In [21]:
train = data[data['Date'] < '2021-11-01']
test =  data[(data['Date'] <= '2021-11-05') & (data['Date'] >= '2021-11-01')]

In [22]:
train.tail()

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi,005930,000660,035420,035720,207940,005935,051910,006400,005380,068270,000270,005490,066570,028260,012330,096770,051900,017670,105560,055550,034730,036570,011200,015760,032830,018260,009150,090430,086790,251270,003670,352820,034020,010950,033780,003490,000810,010130,009540,326030,011170,030200,018880,034220,316140,009830,024110,086280,004020,097950,032640,011780,021240,000720,161390,006800,035250,071050,267250,011070,008930,002790,028050,139480,271560,241560,180640,000100,069500,000120,078930,003410,029780,028670,004990,005830,016360,128940,005387,298020,006360,020150,007070,008770,005940,112610,042660,307950,006280,008560,019170,010620,036460,047050,047810,138040,336260,081660,272210,023530,039490,047040,012750,026960,010060,088350,014680,002380,051915,000060,285130,204320,030000,282330,009240,000990,298050,004170,064350,005385,011210,012510,012450,004800,138930,153130,000080,006260,001450,000880,001440,252670,120110,017800,102110,003090,052690,011000,294870,001230,004370,073240,007310,051600,010120,069620,069960,042700,004000,102780,336370,111770,006650,000240,010780,016380,000150,009900,013890,005850,001740,139130,181710,175330,005250,185750,003000,051905,330590,192820,214980,122630,031430,273130,032350,066575,363280,298000,278530,000670,292150,161890,005300,004490,214320,091990,293490,247540,068760,263750,036490,028300,096530,035760,196170,066970,145020,253450,058470,357780,240810,064760,278280,032500,095700,108320,237690,007390,098460,048260,178920,034230,005290,025900,067160,078340,030190,056190,268600,041510,085660,035900,214150,200130,039030,213420,141080,060250,086900,206650,235980,323990,022100,140410,036540,000250,048410,086450,084850,046890,084370,950130,069080,299660,039200,166090,144510,122870,218410,067630,112040,215200,084990,298380,151910,194480,025980,064550,003380,215000,033640,078600,217270,089980,214450,140860,230360,027360,222800,082270,131290,088800,074600,293780,185490,178320,036830,084110,151860,092040,131970,048530,039840,290650,041960,065660,214370,272290,183490,035080,036930,101490,294090,053800,256840,078130,205470,058820,243070,299900,222080,052400,090460,183300,104480,033290,290510,041190,226950,092190,100090,319660,035600,038500,042000,030520,053030,267980,137400,068240,216080,013120,095660,060150,061970,095610,089970,036810,348210,330860,263720,143240,091700,067000,204270,032190,131370,321550,347860,003800,032300,122450,206640,015750,060720,043150,006730,089030,348150,319400,232140,083790,299030,049070,067310,036030,031390,121800,033310,086390,078020,102710,314130,035890,032620,220630,064260,287410,110790,099320
210,2021-10-25,0,43,994.31,15226.7,4566.48,35743.78,3020.54,70200.0,100000.0,402500.0,126000.0,877000.0,63800.0,821000.0,733000.0,211000.0,216500.0,84300.0,313000.0,124500.0,120000.0,268500.0,256500.0,1383000.0,309500.0,59100.0,40500.0,254000.0,622000.0,29500.0,23000.0,70400.0,157000.0,155000.0,192500.0,47300.0,127000.0,150500.0,326000.0,25750.0,107000.0,82500.0,31400.0,248000.0,572000.0,101500.0,97000.0,240000.0,31300.0,14900.0,17800.0,13000.0,43850.0,11500.0,173000.0,45350.0,396000.0,14850.0,179500.0,83300.0,51600.0,43150.0,8870.0,28900.0,88000.0,65000.0,200500.0,59300.0,52700.0,25250.0,168500.0,124000.0,38750.0,57300.0,61200.0,39535.0,150000.0,44350.0,7630.0,35400.0,6200.0,34050.0,66400.0,48650.0,273000.0,98600.0,624000.0,41600.0,105500.0,32500.0,91600.0,13250.0,70200.0,24950.0,121500.0,252000.0,4890.0,58600.0,72000.0,44200.0,22150.0,31350.0,32900.0,50900.0,39000.0,16650.0,106500.0,108000.0,6610.0,84700.0,34800.0,139000.0,3580.0,339000.0,348500.0,390500.0,28850.0,190500.0,63500.0,24600.0,170500.0,116500.0,55900.0,755000.0,260500.0,23600.0,98200.0,87800.0,100000.0,46650.0,105500.0,9170.0,102980.0,37200.0,65800.0,28400.0,34450.0,2330.0,2225.0,91300.0,48300.0,39540.0,33150.0,72900.0,26750.0,26150.0,17600.0,291000.0,5530.0,487000.0,44200.0,62600.0,141500.0,86400.0,30650.0,87000.0,9900.0,83300.0,46050.0,202000.0,17250.0,45850.0,11700.0,10

In [23]:
test

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi,005930,000660,035420,035720,207940,005935,051910,006400,005380,068270,000270,005490,066570,028260,012330,096770,051900,017670,105560,055550,034730,036570,011200,015760,032830,018260,009150,090430,086790,251270,003670,352820,034020,010950,033780,003490,000810,010130,009540,326030,011170,030200,018880,034220,316140,009830,024110,086280,004020,097950,032640,011780,021240,000720,161390,006800,035250,071050,267250,011070,008930,002790,028050,139480,271560,241560,180640,000100,069500,000120,078930,003410,029780,028670,004990,005830,016360,128940,005387,298020,006360,020150,007070,008770,005940,112610,042660,307950,006280,008560,019170,010620,036460,047050,047810,138040,336260,081660,272210,023530,039490,047040,012750,026960,010060,088350,014680,002380,051915,000060,285130,204320,030000,282330,009240,000990,298050,004170,064350,005385,011210,012510,012450,004800,138930,153130,000080,006260,001450,000880,001440,252670,120110,017800,102110,003090,052690,011000,294870,001230,004370,073240,007310,051600,010120,069620,069960,042700,004000,102780,336370,111770,006650,000240,010780,016380,000150,009900,013890,005850,001740,139130,181710,175330,005250,185750,003000,051905,330590,192820,214980,122630,031430,273130,032350,066575,363280,298000,278530,000670,292150,161890,005300,004490,214320,091990,293490,247540,068760,263750,036490,028300,096530,035760,196170,066970,145020,253450,058470,357780,240810,064760,278280,032500,095700,108320,237690,007390,098460,048260,178920,034230,005290,025900,067160,078340,030190,056190,268600,041510,085660,035900,214150,200130,039030,213420,141080,060250,086900,206650,235980,323990,022100,140410,036540,000250,048410,086450,084850,046890,084370,950130,069080,299660,039200,166090,144510,122870,218410,067630,112040,215200,084990,298380,151910,194480,025980,064550,003380,215000,033640,078600,217270,089980,214450,140860,230360,027360,222800,082270,131290,088800,074600,293780,185490,178320,036830,084110,151860,092040,131970,048530,039840,290650,041960,065660,214370,272290,183490,035080,036930,101490,294090,053800,256840,078130,205470,058820,243070,299900,222080,052400,090460,183300,104480,033290,290510,041190,226950,092190,100090,319660,035600,038500,042000,030520,053030,267980,137400,068240,216080,013120,095660,060150,061970,095610,089970,036810,348210,330860,263720,143240,091700,067000,204270,032190,131370,321550,347860,003800,032300,122450,206640,015750,060720,043150,006730,089030,348150,319400,232140,083790,299030,049070,067310,036030,031390,121800,033310,086390,078020,102710,314130,035890,032620,220630,064260,287410,110790,099320
215,2021-11-01,0,44,998.57,15595.9,4613.67,35913.68,2978.94,69900.0,106500.0,408500.0,125500.0,874000.0,65200.0,832000.0,723000.0,208000.0,205500.0,84300.0,296500.0,120500.0,115000.0,253000.0,239500.0,1200000.0,309500.0,56800.0,38300.0,243000.0,648000.0,26750.0,22650.0,67900.0,157000.0,158500.0,191000.0,45200.0,127000.0,144500.0,348500.0,24050.0,101500.0,82000.0,30450.0,235000.0,528000.0,104500.0,95200.0,228000.0,30850.0,14500.0,19650.0,13350.0,38950.0,11200.0,168500.0,44100.0,387500.0,14400.0,171000.0,79100.0,51300.0,41750.0,8790.0,27900.0,86800.0,62800.0,213500.0,58700.0,50600.0,24650.0,170000.0,120000.0,38800.0,55500.0,60000.0,38985.0,146000.0,42500.0,8090.0,34300.0,6080.0,33050.0,59900.0,47400.0,270000.0,97300.0,597000.0,41850.0,97600.0,32800.0,80600.0,13050.0,68400.0,25850.0,116500.0,279000.0,4725.0,49700.0,78900.0,43600.0,21200.0,30050.0,33350.0,51700.0,36650.0,16150.0,109500.0,105000.0,6250.0,85400.0,34150.0,126500.0,3490.0,323000.0,327000.0,375500.0,27850.0,179500.0,63900.0,23700.0,167000.0,99700.0,57300.0,695000.0,238000.0,21950.0,96400.0,79800.0,84600.0,43850.0,102000.0,8740.0,102925.0,35200.0,62100.0,26500.0,32750.0,2285.0,2285.0,83700.0,47650.0,39020.0,32100.0,70100.0,30050.0,25500.0,17000.0,286000.0,5400.0,482500.0,42250.0,58900.0,144500.0,80900.0,31400.0,84000.0,9740.0,80500.0,43650.0,184000.0,16600.0,45600.0,11350.0,10300

# Pycaret으로 각 특성 예측

In [24]:
train.columns[0:8]

Index(['Date', 'weekday', 'weeknum', 'kosdaq', 'nasdaq', 'sp500', 'dow',
       'kospi'],
      dtype='object')

In [25]:
features = list(train.columns)
type(features)

list

In [26]:
end_features = features[:3]
end_features

['Date', 'weekday', 'weeknum']

In [27]:
type(end_features[0])

str

In [28]:
test[end_features]

,Date,weekday,weeknum
215,2021-11-01,0,44
216,2021-11-02,1,44
217,2021-11-03,2,44
218,2021-11-04,3,44
219,2021-11-05,4,44


### 테스트셋의 5가지 특성 예측 하기

In [29]:
for i in range(5):
        model = setup(train, target = features[i+3], ignore_features = features[i+4:], silent=True)
        tuned_model = tune_model(compare_models(sort='MAPE', n_select=1))
        tuned_model_ = finalize_model(tuned_model)
        end_features = features[:i+3]
        pred = predict_model(tuned_model_, test[end_features])
        test[features[i+3]] = pred['Label']

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,10.4836,163.7735,12.7974,0.9807,0.0040,0.0033
1,23.0771,742.8289,27.2549,0.8844,0.0088,0.0074
2,17.4629,427.0089,20.6642,0.9600,0.0067,0.0056
3,17.2098,423.8175,20.5868,0.9465,0.0066,0.0055
4,16.5277,470.0221,21.6800,0.9648,0.0070,0.0053
5,16.2931,497.9409,22.3146,0.9240,0.0072,0.0052
6,11.5963,219.2590,14.8074,0.9678,0.0046,0.0036
7,21.6635,697.7261,26.4145,0.9387,0.0087,0.0071
8,14.2672,410.2976,20.2558,0.9561,0.0065,0.0046
9,17.4196,522.7201,22.8631,0.9472,0.0072,0.0055


In [30]:
test

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi,005930,000660,...,078020,102710,314130,035890,032620,220630,064260,287410,110790,099320
215,2021-11-01,0,44,1012.535450,15170.686752,4528.091834,34844.781250,3103.864746,69900.0,106500.0,...,9030.0,28950.0,36600.0,1845.0,8050.0,4930.0,8970.0,7480.0,41600.0,50300.0
216,2021-11-02,1,44,1011.437549,15223.466707,4560.931247,35150.609375,3104.271729,71500.0,107500.0,...,9060.0,29150.0,38000.0,1875.0,8090.0,4965.0,9050.0,7430.0,41650.0,49950.0
217,2021-11-03,2,44,1010.168437,15179.775583,4517.739981,34686.503906,3082.286133,70400.0,105500.0,...,9040.0,29000.0,38650.0,1830.0,7900.0,4980.0,9080.0,7110.0,42000.0,48850.0
218,2021-11-04,3,44,1010.553262,15286.072958,4560.671574,35069.941406,3078.474609,70600.0,106000.0,...,9020.0,28750.0,39750.0,1815.0,7870.0,4975.0,8100.0,6990.0,42100.0,47350.0
219,2021-11-05,4,44,1011.482073,15258.515205,4571.155994,35223.636719,3094.673340,70200.0,107000.0,...,8940.0,28650.0,38400.0,1800.0,7670.0,5010.0,8480.0,6780.0,41850.0,45550.0


In [31]:
train.tail()

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi,005930,000660,...,078020,102710,314130,035890,032620,220630,064260,287410,110790,099320
210,2021-10-25,0,43,994.31,15226.7,4566.48,35743.78,3020.54,70200.0,100000.0,...,9070.0,29100.0,36600.0,1865.0,7880.0,4620.0,7690.0,7200.0,43450.0,50200.0
211,2021-10-26,1,43,1011.76,15235.7,4574.79,35755.83,3049.08,71100.0,102000.0,...,9230.0,29950.0,38000.0,1930.0,7960.0,4750.0,7530.0,7180.0,44450.0,49550.0
212,2021-10-27,2,43,1008.95,15235.8,4551.68,35491.48,3025.49,70100.0,101500.0,...,9210.0,29350.0,37450.0,1900.0,7970.0,4840.0,8080.0,7270.0,43750.0,48200.0
213,2021-10-28,3,43,1000.13,15448.1,4596.42,35729.89,3009.55,70700.0,106500.0,...,9140.0,29700.0,38250.0,1860.0,7830.0,4860.0,7620.0,7220.0,42950.0,49750.0
214,2021-10-29,4,43,992.33,15498.4,4605.38,35819.59,2970.68,69800.0,103000.0,...,9050.0,28900.0,37200.0,1840.0,7670.0,4885.0,8220.0,7120.0,42850.0,50200.0


In [32]:
target_feature = features[9]
basic_features = features[:8] + [target_feature]
basic_features

['Date',
 'weekday',
 'weeknum',
 'kosdaq',
 'nasdaq',
 'sp500',
 'dow',
 'kospi',
 '000660']

In [33]:
ignore_features = features
for col in basic_features:
    ignore_features.remove(col)
len(ignore_features)

369

In [34]:
type(ignore_features)

list

In [37]:
test

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi,005930,000660,...,078020,102710,314130,035890,032620,220630,064260,287410,110790,099320
215,2021-11-01,0,44,1012.535450,15170.686752,4528.091834,34844.781250,3103.864746,69900.0,106500.0,...,9030.0,28950.0,36600.0,1845.0,8050.0,4930.0,8970.0,7480.0,41600.0,50300.0
216,2021-11-02,1,44,1011.437549,15223.466707,4560.931247,35150.609375,3104.271729,71500.0,107500.0,...,9060.0,29150.0,38000.0,1875.0,8090.0,4965.0,9050.0,7430.0,41650.0,49950.0
217,2021-11-03,2,44,1010.168437,15179.775583,4517.739981,34686.503906,3082.286133,70400.0,105500.0,...,9040.0,29000.0,38650.0,1830.0,7900.0,4980.0,9080.0,7110.0,42000.0,48850.0
218,2021-11-04,3,44,1010.553262,15286.072958,4560.671574,35069.941406,3078.474609,70600.0,106000.0,...,9020.0,28750.0,39750.0,1815.0,7870.0,4975.0,8100.0,6990.0,42100.0,47350.0
219,2021-11-05,4,44,1011.482073,15258.515205,4571.155994,35223.636719,3094.673340,70200.0,107000.0,...,8940.0,28650.0,38400.0,1800.0,7670.0,5010.0,8480.0,6780.0,41850.0,45550.0


In [53]:
test.columns[150:200]

Index(['007310', '051600', '010120', '069620', '069960', '042700', '004000',
       '102780', '336370', '111770', '006650', '000240', '010780', '016380',
       '000150', '009900', '013890', '005850', '001740', '139130', '181710',
       '175330', '005250', '185750', '003000', '051905', '330590', '192820',
       '214980', '122630', '031430', '273130', '032350', '066575', '363280',
       '298000', '278530', '000670', '292150', '161890', '005300', '004490',
       '214320', '091990', '293490', '247540', '068760', '263750', '036490',
       '028300'],
      dtype='object')

In [38]:
test2 = test.copy()

In [40]:
test2.columns

Index(['Date', 'weekday', 'weeknum', 'kosdaq', 'nasdaq', 'sp500', 'dow',
       'kospi', '005930', '000660',
       ...
       '078020', '102710', '314130', '035890', '032620', '220630', '064260',
       '287410', '110790', '099320'],
      dtype='object', length=378)

In [42]:
test2 = test2[['Date', 'weekday', 'weeknum', 'kosdaq', 'nasdaq', 'sp500', 'dow', 'kospi']]
test2.head()

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi
215,2021-11-01,0,44,1012.535450,15170.686752,4528.091834,34844.781250,3103.864746
216,2021-11-02,1,44,1011.437549,15223.466707,4560.931247,35150.609375,3104.271729
217,2021-11-03,2,44,1010.168437,15179.775583,4517.739981,34686.503906,3082.286133
218,2021-11-04,3,44,1010.553262,15286.072958,4560.671574,35069.941406,3078.474609
219,2021-11-05,4,44,1011.482073,15258.515205,4571.155994,35223.636719,3094.673340


In [63]:
stock_pred_dict = {}
features = list(train.columns)
for i in range(3):
    # 타겟 특성
    target_feature = features[i+8]

    # pycaret으로 훈련, 예측
    model = setup(train, target = target_feature, ignore_features = features[i+9:], silent=True)
    tuned_model = tune_model(compare_models(sort='MAPE', n_select=1))
    tuned_model_ = finalize_model(tuned_model)
    # 기본 3개 + 외부데이터 5개로 예측
    pred = predict_model(tuned_model_, test[features[:i+8]])
    # 예측값 test셋에 종목에 기록
    test[target_feature] = pred['Label']
    
    # test셋의 종목 종가 dict에 종목 코드로 기록
    stock_pred_dict[target_feature] = test[target_feature]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,7698.3974,83623990.0813,9144.6154,0.9076,0.0239,0.0198
1,6140.7692,71750089.7706,8470.5425,0.9487,0.0225,0.0159
2,8453.4509,124507248.8178,11158.2816,0.8713,0.0318,0.0230
3,5805.9188,57094432.6387,7556.0858,0.9038,0.0194,0.0147
4,8622.7777,98011785.2259,9900.0902,0.9469,0.0288,0.0243
5,7162.4146,92260859.5147,9605.2517,0.9061,0.0251,0.0190
6,8914.5299,198796502.5117,14099.5214,0.8817,0.0350,0.0230
7,6422.0620,77649235.2441,8811.8803,0.9298,0.0214,0.0158
8,5561.4202,48952406.4019,6996.5996,0.9686,0.0213,0.0163
9,5396.0691,43635287.4405,6605.7011,0.9440,0.0163,0.0135


In [64]:
stock_pred_dict

{'005930': 215    74627.246320
 216    73965.025190
 217    71580.217211
 218    71212.728797
 219    73079.304257
 Name: 005930, dtype: float64,
 '000660': 215    109583.694160
 216    108889.693876
 217    102612.827008
 218    104779.673691
 219    109958.428158
 Name: 000660, dtype: float64,
 '035420': 215    408758.653846
 216    409221.153846
 217    406460.096154
 218    409212.500000
 219    409928.365385
 Name: 035420, dtype: float64}

In [65]:
len(stock_pred_dict)

3

In [ ]:
stock_pred = pd.DataFrame(stock_pred_dict)
stock_pred

In [ ]:
stock_pred.columns

In [ ]:
stock_pred * 2

In [ ]:
type(stock_pred['005930'])

In [ ]:
final_stock_pred = pd.concat([stock_pred, stock_pred])
final_stock_pred

In [ ]:
final_stock_pred = final_stock_pred.reset_index()
final_stock_pred

In [ ]:
final_stock_pred = final_stock_pred.drop('index',axis=1)
final_stock_pred

In [ ]:
final_stock_pred.columns

# 제출 파일 만들기

In [ ]:
sample_name = 'sample_submission.csv'
sub = pd.read_csv(os.path.join(path,sample_name))
sub

In [ ]:
for code in final_stock_pred.columns:
    sub[code] = final_stock_pred[code]

In [ ]:
sub

In [ ]:
sub.to_csv('sub01_cat01.csv', index=False)